In [ ]:
!pip3 install openai langchain pinecone-client tiktoken numpy pandas flask firebase-admin firebase-auth streamlit pytest python-dotenv pylint
!pip install promptlayer
!pip install openai
!pip install openai >> null
!pip install beautifulsoup4
!pip install google
!pip install beautifulsoup4
!pip install python-docx
!pip install spacy spacy-llm
!pip install sumy

In [7]:

%%capture
import os
import time
import csv
import sqlite3
import pickle
import json
import openai
import sys
import smtplib, ssl
import time
import imaplib
import email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from openai import OpenAI
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import imaplib
import smtplib
import email
import ssl
import datetime
from docx import Document
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

class MailBot:
    def __init__(self, mail_host, password, sender_email, sent_folder, inbox_folder, IMAP_SSL, SMTP_SSL):
        self.mail_host = mail_host
        self.password = password
        self.sender_email = sender_email
        self.sent_folder = sent_folder
        self.inbox_folder = inbox_folder
        self.IMAP_SSL = IMAP_SSL
        self.SMTP_SSL = SMTP_SSL

    def reply_to_emails(self, unread_messages: bool):
        # Connect to the mailbox
        unread_mails = []
        mail = imaplib.IMAP4_SSL(self.mail_host)
        mail.login(self.sender_email, self.password)
        mail.select(self.inbox_folder)

        # Search for unread emails in the mailbox
        status, messages = mail.search(None, 'UNSEEN' if unread_messages else 'ALL')
        message_ids = messages[0].split()

        for msg_id in message_ids:
            # Fetch the email
            status, data = mail.fetch(msg_id, '(RFC822)')
            raw_email = data[0][1]
            email_message = email.message_from_bytes(raw_email)

            # Extract email details
            sender = email_message['From']
            if sender.endswith("howard.edu>"):
              old_subject = email_message['Subject']
              parsing_subject = old_subject.split(':')[-1]
              subject = f'REPLY: {parsing_subject}'
              date = email_message['Date']

              # Get the email body content
              body = self.get_email_body(email_message, old_subject)
              body_str = str(body, encoding='utf-8')

              unread_mails.append(body_str)
            mail.store(msg_id, '-FLAGS', '\\Seen')   #just to mark the email as unread.

        mail.logout()
        return unread_mails

    def get_email_body(self, email_message, subject):
        body = ""

        if email_message.is_multipart():
            for part in email_message.walk():
                content = part.get_content_type()
                disposition = str(part.get('Content-Disposition'))
                if content == 'text/plain' and 'attachment' not in disposition:
                    body = part.get_payload(decode=True)
                    break
        else:
            body = email_message.get_payload(decode=True)

        return body

def start_email_bot():
  # Email Variables
  new_mail_host = "smtp-mail.outlook.com"
  new_password = "Testuser@2930"
  new_your_email = "hbcu_test@outlook.com"
  new_sent_folder = 'SENT'
  new_inbox_folder = 'INBOX'
  new_SMTP_SSL = 587
  new_IMAP_SSL = 993

  # Instance of bot class
  mail_operator = MailBot(
      new_mail_host,
      new_password,
      new_your_email,
      new_sent_folder,
      new_inbox_folder,
      new_IMAP_SSL,
      new_SMTP_SSL
  )

  # Here we read all messages
  data_list = mail_operator.reply_to_emails(unread_messages=True)
  data_text = " ".join(data_list)
  return data_text

In [3]:
email_data = start_email_bot()
print(email_data)

Dear Prospective Bison,

Greetings from Howard University! We hope this message finds you well and
filled with excitement about the possibilities that lie ahead in your
academic journey.

As the deadline for submitting your application to Howard University draws
near, we wanted to reach out and remind you that time is of the essence.
The application deadline is fast approaching, and we don't want you to miss
out on the chance to join our vibrant and dynamic community.

Here at Howard, we believe in the power of education to transform lives and
shape futures. Our institution is not just a place of learning; it's a
place where dreams are nurtured, talents are honed, and aspirations are
realized. As a historically Black university with a rich legacy of
excellence, we take pride in fostering a supportive and inclusive
environment where every student can thrive and succeed.

Whether you're drawn to our renowned programs in the arts, sciences,
business, or any of our other academic disciplin

In [9]:
summary_text = summarize(email_data)
print(summary_text)


Summary:
We hope this message finds you well and filled with excitement about the possibilities that lie ahead in your academic journey.
We are eagerly awaiting your application and can't wait to welcome you into the Howard University family.
- Students who have jobs or internships are able to select classes that work with their outside schedules.
*Please also note that the course waitlist option is no longer available once a given semester has commenced.
All students are encouraged to register for alternate course options with the assistance of their assigned academic adviser.
Please note that students on the waitlist will be given priority consideration for course override approval.
We also invite everyone who can't attend in person to rock their Bison spirit wherever they are worldwide.
We will also celebrate Howard alumnus, Black Arts Movement pioneer, and noted playwright Richard E. Wesley with a keynote interview.
* "One of my favorite memories is being a site coordinator for Ho

In [4]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def summarize(input_text):
  # Parse the input text
  parser = PlaintextParser.from_string(input_text, Tokenizer("english"))

  # Create an LSA summarizer
  summarizer = LsaSummarizer()

  # Generate the summary
  summary = summarizer(parser.document, sentences_count=10)  # You can adjust the number of sentences in the summary

  # Output the summary
  print("\nSummary:")
  for sentence in summary:
      print(sentence)



In [5]:
def summarize1(text, per):
    nlp = spacy.load('en_core_web_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    print("final_summary", final_summary)
    summary=''.join(final_summary)
    print(text)
    return summary